In [1]:
import pandas as pd
import numpy as np
import os

## function testing

In [2]:
def rename_duplicates(Gemeinde:str,Landkreis:str,name_duplicates:list) -> str:
    if Gemeinde in name_duplicates:
        return f'{Gemeinde} ({Landkreis})'
    else:
        return Gemeinde


def first_setup() -> pd.DataFrame:
    ############# MOBILE:
    mobile = pd.read_excel('../data/external/bba_06_2023.xlsx', sheet_name='Fläche', skiprows=2, header=1)
    mobile = mobile[(mobile['Land']=='Freistaat Bayern') & (mobile['Verwaltungsebene']!='2 - Land') & (~mobile['Name'].str.contains('Bezirk'))].reset_index(drop=True)

    # Landkreis-Code auf einheitliches Format bringen
    mobile['code_fill'] = mobile['AGS'].apply(lambda x: int(str(x)[1:].ljust(6, '0')))
    mobile['Name'] = mobile['Name'].apply(lambda x: x.replace('Gemeinde ','').replace('Landkreis ',''))

    # Alle Doppelungen löschen -> Kreisfreie Städte tauchen nur in der Liste für Gemeinde/Städte auf
    mobile = mobile.sort_values('Verwaltungsebene',ascending=False,ignore_index=True)
    mobile.drop_duplicates('code_fill',ignore_index=True,inplace=True)

    # Jeder Gemeinde wird ihr Landkreis zugeordnen
    LK = mobile[mobile['Verwaltungsebene']=='3 - Kreis']
    LK_dict = LK[['code_fill','Name']].set_index('code_fill').to_dict()['Name']
    mobile['LK_code'] = mobile['code_fill'].apply(lambda x: int(str(x)[:3]+'000'))
    mobile['Landkreis'] = mobile['LK_code'].map(LK_dict)

    # Suche nach allen mehrfach benannten Gemeinden und bei Bedarf Hinzufügen des Landkreises
    name_counts = mobile[mobile['Verwaltungsebene']=='4 - Gemeinde']['Name'].value_counts()
    name_duplicates = name_counts[name_counts>1].keys()
    mobile['Name'] = mobile.apply(lambda x: rename_duplicates(x.Name,x.Landkreis,name_duplicates),axis=1)

    # Datenfilter
    mobile = mobile[['code_fill','Name','Verwaltungsebene','2G','4G','5G']]

    return mobile

In [3]:
def mobile_data(code, mobile):
    mobile_i = mobile[mobile['code_fill']==code].reset_index(drop=True)
    
    export_mobile = {'2G':round(mobile_i['2G'][0],1), 
                     '4G':round(mobile_i['4G'][0],1), 
                     '5G':round(mobile_i['5G'][0],1), 
                     'link':'https://gigabitgrundbuch.bund.de/GIGA/DE/MobilfunkMonitoring/Downloads/start.html'}
    return export_mobile

In [4]:
mobile = first_setup()

In [269]:
Landkreise = mobile[mobile['Verwaltungsebene']=='3 - Kreis'][['code_fill','Name']].set_index('code_fill').to_dict()['Name']
Gemeinden = mobile[mobile['Verwaltungsebene']=='4 - Gemeinde'][['code_fill','Name']].set_index('code_fill').to_dict()['Name']

In [275]:
LK_codes = list(Landkreise.keys())
town_codes = list(Gemeinden.keys())

In [295]:
errors = []
for code in LK_codes:
    try:
        result = mobile_data(code, mobile)
    except:
        errors.append(code)

for code in town_codes:
    try:
        result = mobile_data(code, mobile)
    except:
        errors.append(code)        

errors = list(set(errors))

mobile[mobile['code_fill'].isin(errors)]

,code_fill,Name,Verwaltungsebene,2G,4G,5G
